In [110]:
# Заранее извиняюсь за орфографические ошибки и опечатки
import random as rnd
import numpy as np
import math
import time

In [117]:
# Начальные условия для поиска минимум заданной функции посредством генетического алгоритма

N = 25 # 2^(N + 1) - размер сетки 
ITER = 600 # количество итераций работы алгоритма
SIZE = 80 # размер популяции
ALPHA = 0.5 # доля популяции, которая остается при селекции(всегда будет хотя бы одна, но всегда хотя бы одна вымрет)
GAMMA = 0.1 # доля особей, которая мутирует в конце итерации(всегда будет хотя бы одна)
# размер зоны поиска минимума по x и y соответственно  
BX = -1
EX = 1
BY = -1
EY = 1

# функция, которая подлежит исследованию (4 + math.pow((x*x + y*y)**2, 1/3.0) - минимум в (0,0))
def f(x, y):
    return 4 + math.pow((x*x + y*y)**2, 1/3.0)


# некоторые начальные подсчеты
REST = int(ALPHA * SIZE)
if REST == 0 : 
    REST = 1
if REST == SIZE :
    REST = SIZE - 1
MUT = int(GAMMA * SIZE)
if MUT == 0 : 
    MUT = 1

X, Y = np.linspace(BX, EX, num=2**(N+1)+1), np.linspace(BY, EY, num=2**(N+1)+1)

In [118]:
# перевод числа в двоичную систему
def my_bin(x):
    answ = []
    while (x > 0) or (len(answ) < N + 1) :
        answ.append(x % 2)
        x = x // 2
    answ.reverse()
    return answ

# обратный перевод
def rebin(x):
    answ = ''.join([str(y) for y in x])
    return int(answ, 2)

In [119]:
# Функции жизненного цикла популяции

# функция, генерирующая стартовую популяцию
# особь популяции - [геном, положение по иксу, положение по игрику, значение функции в этой точке, флаг модификации генома]
def genPopulation():
    population = []
    for _ in range(SIZE) :
        tmp1, tmp2 = rnd.randint(0, 2**(N+1) - 1), rnd.randint(0, 2**(N+1) - 1)
        population.append([ my_bin(tmp1) + my_bin(tmp2), X[tmp1], Y[tmp2], f(X[tmp1],Y[tmp2]), False])
    population.sort(key=lambda arg : arg[3])
    return population

# скрещивание самых удачных кандидатов
# из удачной части берутся 2 случайных кандидата(это может быть тот же самый кандидат, тк бывают случаи, что удачной
# считается часть из одного индивида) и от одного берется иксовая составляющая, от другого игриковая
# они совмещаются и получается новая особь
def crossing(population):
    for i in range(SIZE - REST):
        mom, dad = rnd.randint(0, REST - 1), rnd.randint(0, REST - 1)
        mom_ch, dad_ch = population[mom][0][:N + 1], population[dad][0][N + 1:]
        population[i + REST - 1][0] = mom_ch + dad_ch
        population[i + REST - 1][4] = True
    return population

# случайные мутации
# у нескольких особей один из генов (один бит) меняется на противоположный
# одна и та же особь может быть выбрана несколько раз
def mutation(population):
    for _ in range(MUT) :
        ind = rnd.randint(0, SIZE-1)
        gen = rnd.randint(0, (N + 1) * 2 - 1)
        population[ind][0][gen] = (population[ind][0][gen] + 1 ) % 2
        population[ind][4] = True 
    return population

# пересчет значений у новых и мутировавших экземпляров
# обход популяции и проверка флагов модификации, если он положителен, то производится пересчет
def recalc(population) :
    for i in population:
        if i[4] :
            tmp1, tmp2 = rebin(i[0][:N + 1]), rebin(i[0][N + 1:])
            i[1], i[2], i[3], i[4] = X[tmp1], Y[tmp2], f(X[tmp1], Y[tmp2]), False
    return population

In [121]:
start = int(time.time() * 1000)
population = genPopulation()
for _ in range(ITER):
    population = crossing(population)
    population = mutation(population)
    population = recalc(population)
    population.sort(key=lambda arg : arg[3])
print(int(time.time() * 1000) - start)
population[0][1:4]

526


[-2.9802322387695312e-08, -2.9802322387695312e-08, 4.000000000146674]

In [81]:
f(0, 1)


-2